In [0]:
yellow_table=spark.read.csv("/Volumes/azuredatabricks_2000/default/yellow_taxi/yellow_tripdata_2016-01.csv",inferSchema=True,header=True)

In [0]:
yellow_table.coalesce(1).write.parquet("/Volumes/azuredatabricks_2000/default/yellow_taxi/yellowparquet")

In [0]:
# yellow_table.write.parquet("")

In [0]:
yellow_table.schema

StructType([StructField('VendorID', IntegerType(), True), StructField('tpep_pickup_datetime', TimestampType(), True), StructField('tpep_dropoff_datetime', TimestampType(), True), StructField('passenger_count', IntegerType(), True), StructField('trip_distance', DoubleType(), True), StructField('pickup_longitude', DoubleType(), True), StructField('pickup_latitude', DoubleType(), True), StructField('RatecodeID', IntegerType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('dropoff_longitude', DoubleType(), True), StructField('dropoff_latitude', DoubleType(), True), StructField('payment_type', IntegerType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True)])

In [0]:
%sql 
create live table yellowtaxi_bronze(
  filename  string,
  createdon timestamp,
VendorID integer,
tpep_pickup_datetime timestamp,
tpep_dropoff_datetime timestamp,
passenger_count integer,
trip_distance double,
pickup_longitude double,
pickup_latitude double,
RatecodeID integer,
store_and_fwd_flag string,
dropoff_longitude double,
dropoff_latitude double,
payment_type integer,
fare_amount double,
extra double,
mta_tax double,
tip_amount double,
tolls_amount double,
improvement_surcharge double,
total_amount double
  
)using delta partitioned by (VendorID)
as select * ,input_file_name() as filename,current_timestamp() as createdon
from parquet.`/Volumes/azuredatabricks_2000/default/yellow_taxi/yellowparquet/`

Name,Type
filename,string
createdon,timestamp
VendorID,int
tpep_pickup_datetime,timestamp
tpep_dropoff_datetime,timestamp
passenger_count,int
trip_distance,double
pickup_longitude,double
pickup_latitude,double
RatecodeID,int


In [0]:
yellow_table.select("passenger_count").distinct().show()

+---------------+
|passenger_count|
+---------------+
|              1|
|              6|
|              3|
|              5|
|              9|
|              4|
|              8|
|              7|
|              2|
|              0|
+---------------+



In [0]:
live

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-237031983626651>, line 1
----> 1 live.tables

NameError: name 'live' is not defined

In [0]:
%sql 
create live table yellowtaxi_silver(
  filename  string,
  createdon timestamp,
VendorID integer,
tpep_pickup_datetime timestamp,
tpep_dropoff_datetime timestamp,
passenger_count integer,
trip_distance double,
pickup_longitude double,
pickup_latitude double,
RatecodeID integer,
payment_type integer,
fare_amount double,
total_amount double,

pickupYear Int generated always as (YEAR (tpep_pickup_datetime)),
pickupMonth Int generated always as (MONTH (tpep_pickup_datetime)),
pickupDay Int generated always as (DAY(tpep_pickup_datetime)),

 constraint ValidTotalAmount Expect (total_amount is not null and total_amount > 0) on violation drop row,
constraint ValidDistance Expect (trip_distance is not null and trip_distance > 0) on violation drop row,
  constraint ValidCount Expect (passenger_count > 0) on violation  drop row ,
  constraint ValidVendorId Expect (VendorID is not null and VendorID > 0) on violation  fail update 



)using delta partitioned by (passenger_count)
as select 
VendorID ,
tpep_pickup_datetime ,
tpep_dropoff_datetime ,
passenger_count ,
trip_distance ,
pickup_longitude ,
pickup_latitude ,
RatecodeID ,
payment_type ,
fare_amount ,
total_amount ,
current_timestamp() as createdon
from live.yellowtaxi_bronze

Name,Type
filename,string
createdon,timestamp
VendorID,int
tpep_pickup_datetime,timestamp
tpep_dropoff_datetime,timestamp
passenger_count,int
trip_distance,double
pickup_longitude,double
pickup_latitude,double
RatecodeID,int


In [0]:
%sql 
create live table yellowtaxi_gold
as 
select RateCodeID,passenger_count,
count(*) as totalRide,
sum(trip_distance) as total_distance ,
sum(total_amount) as totalAmount
from live.yellowtaxi_silver
group by RateCodeID,passenger_count




Name,Type
RateCodeID,int
passenger_count,int
totalRide,bigint
total_distance,double
totalAmount,double
